In [ ]:
import sqlite3

In [3]:
connection = sqlite3.connect("Agentic_AI.db")

In [4]:
connection

In [5]:

table_creation_query="""
CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    hire_date TEXT NOT NULL,
    salary REAL NOT NULL
);
"""

In [ ]:
table_creation_query2="""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    phone TEXT
);
"""


In [7]:

table_creation_query3="""
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date TEXT NOT NULL,
    amount REAL NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
"""

In [8]:
cursor = connection.cursor()

In [9]:
cursor.execute(table_creation_query)
cursor.execute(table_creation_query2)
cursor.execute(table_creation_query3)

In [10]:

insert_query = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?);
"""


insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone)
VALUES (?, ?, ?, ?, ?);
"""

insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?);
"""

In [11]:
employee_data = [
    (1, "Sunny", "Savita", "sunny.sv@abc.com", "2023-06-01", 50000.00),
    (2, "Arhun", "Meheta", "arhun.m@gmail.com", "2022-04-15", 60000.00),
    (3, "Alice", "Johnson", "alice.johnson@jpg.com", "2021-09-30", 55000.00),
    (4, "Bob", "Brown", "bob.brown@uio.com", "2020-01-20", 45000.00)
]

customers_data = [
    (1, "John", "Doe", "john.doe@example.com", "1234567890"),
    (2, "Jane", "Smith", "jane.smith@example.com", "9876543210"),
    (3, "Emily", "Davis", "emily.davis@example.com", "4567891230"),
    (4, "Michael", "Brown", "michael.brown@example.com", "7894561230")
]

orders_data = [
    (1, 1, "2023-12-01", 250.75),
    (2, 2, "2023-11-20", 150.50),
    (3, 3, "2023-11-25", 300.00),
    (4, 4, "2023-12-02", 450.00)
]

In [12]:
cursor.executemany(insert_query,employee_data)
cursor.executemany(insert_query_customers,customers_data)
cursor.executemany(insert_query_orders,orders_data)

In [13]:
connection.commit()

In [18]:
print("Employee Data\n")
data_e = cursor.execute("Select * from employees")
for row in cursor.fetchall():
    print(row)
print("\n Custormer Data\n")
data_c = cursor.execute("Select * from customers")
for row in cursor.fetchall():
    print(row)
print("\nOrders Data\n")
data_o = cursor.execute("Select * from orders")
for row in cursor.fetchall():
    print(row)

Employee Data

(1, 'Sunny', 'Savita', 'sunny.sv@abc.com', '2023-06-01', 50000.0)
(2, 'Arhun', 'Meheta', 'arhun.m@gmail.com', '2022-04-15', 60000.0)
(3, 'Alice', 'Johnson', 'alice.johnson@jpg.com', '2021-09-30', 55000.0)
(4, 'Bob', 'Brown', 'bob.brown@uio.com', '2020-01-20', 45000.0)

 Custormer Data

(1, 'John', 'Doe', 'john.doe@example.com', '1234567890')
(2, 'Jane', 'Smith', 'jane.smith@example.com', '9876543210')
(3, 'Emily', 'Davis', 'emily.davis@example.com', '4567891230')
(4, 'Michael', 'Brown', 'michael.brown@example.com', '7894561230')

Orders Data

(1, 1, '2023-12-01', 250.75)
(2, 2, '2023-11-20', 150.5)
(3, 3, '2023-11-25', 300.0)
(4, 4, '2023-12-02', 450.0)


In [19]:
from langchain_community.utilities import SQLDatabase

In [20]:
db = SQLDatabase.from_uri("sqlite:///Agentic_AI.db")

In [21]:
db

In [22]:
db.dialect

'sqlite'

In [24]:
db.get_usable_table_names()

['customers', 'employees', 'orders']

In [27]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-70b-8192")

In [29]:
llm.invoke("Who is Narendra Modi?")

AIMessage(content="Narendra Damodardas Modi is the 14th and current Prime Minister of India, serving since 2014. He is a member of the Bharatiya Janata Party (BJP) and the Rashtriya Swayamsevak Sangh (RSS), a Hindu nationalist organization.\n\nEarly Life and Career:\n\nModi was born on September 17, 1950, in Vadnagar, Gujarat, India. He grew up in a lower-middle-class family and was the third of six children. He completed his Bachelor's degree in Political Science from the University of Delhi and later earned a Master's degree in Political Science from Gujarat University.\n\nModi joined the RSS in his teenage years and rose through the ranks, eventually becoming a full-time pracharak (worker) in 1972. He worked in various capacities within the RSS and BJP, including as a party organizer and strategist.\n\nModi's entry into electoral politics began in 1987, when he became the BJP's organizational secretary in Gujarat. He played a key role in the BJP's election campaigns in Gujarat and e

In [30]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [32]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [34]:
tools = toolkit.get_tools()

In [35]:
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


In [37]:
list_tables_tool = next((tool for tool in tools if tool.name=="sql_db_list_tables"))
get_schema_tool = next((tool for tool in tools if tool.name=="sql_db_schema"))

In [45]:
list_tables_tool.invoke("")

'customers, employees, orders'

In [55]:
print(get_schema_tool.invoke("employees"))


CREATE TABLE employees (
	emp_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	hire_date TEXT NOT NULL, 
	salary REAL NOT NULL, 
	PRIMARY KEY (emp_id), 
	UNIQUE (email)
)

/*
3 rows from employees table:
emp_id	first_name	last_name	email	hire_date	salary
1	Sunny	Savita	sunny.sv@abc.com	2023-06-01	50000.0
2	Arhun	Meheta	arhun.m@gmail.com	2022-04-15	60000.0
3	Alice	Johnson	alice.johnson@jpg.com	2021-09-30	55000.0
*/


In [50]:
from langchain_core.tools import tool
@tool
def db_query_tool(query:str)->str:
    """
    Execute a SQL query against the database and return the result.
    If the query is invalid or returns no result, an error message will be returned.
    In case of an error, the user is advised to rewrite the query and try again.
    """
    result=db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    return result

In [61]:
db.run("Select * from employees")

"[(1, 'Sunny', 'Savita', 'sunny.sv@abc.com', '2023-06-01', 50000.0), (2, 'Arhun', 'Meheta', 'arhun.m@gmail.com', '2022-04-15', 60000.0), (3, 'Alice', 'Johnson', 'alice.johnson@jpg.com', '2021-09-30', 55000.0), (4, 'Bob', 'Brown', 'bob.brown@uio.com', '2020-01-20', 45000.0)]"

In [60]:
db_query_tool.invoke("Select * from employees")

"[(1, 'Sunny', 'Savita', 'sunny.sv@abc.com', '2023-06-01', 50000.0), (2, 'Arhun', 'Meheta', 'arhun.m@gmail.com', '2022-04-15', 60000.0), (3, 'Alice', 'Johnson', 'alice.johnson@jpg.com', '2021-09-30', 55000.0), (4, 'Bob', 'Brown', 'bob.brown@uio.com', '2020-01-20', 45000.0)]"